# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [6]:
# Start by installing the dependencies
# ! pip install ipyleaflet
!pip install python-frontmatter getorg --upgrade

import frontmatter
import glob
import getorg, re, pathlib
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut
from ipyleaflet import Map, TileLayer

In [2]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [3]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [4]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Unveiling Global Patterns in Taxonomic and Gene Expression Dynamics - Species Abundance Distributions and their Biological Interpretation<br />ICTP; Trieste, Italy Trieste, Friuli-Venezia Giulia, 34121-34151, Italia
Complexity and emergence in marine ecosystems/seascape: theory, mechanisms and data analysis<br />Synergy Summer School 2024; Ischia, Italy Ischia, Napoli, Campania, 80077, Italia
Complexity and emergence in marine ecosystems seascape: theory, mechanisms, and data<br />Synergy Summer School 2025; Ischia, Italy Ischia, Napoli, Campania, 80077, Italia
Delay effects on the stability of large ecosystems<br />Italian Society of Statistical Physics - Young seminars; Padova, Italy Padova, Veneto, Italia
Deviation from neutral species abundance distributions unveils geographical differences in the structure of diatom communities<br />ICTP; Trieste, Italy Trieste, Friuli-Venezia Giulia, 34121-34151, Italia
Modeling bio-geographies – a conceptual approach<br />AtlantECO and CEODOS; S

In [5]:
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'

In [8]:
def set_bicolor_tiles(folder="talkmap", theme="dark"):
    tiles = {
        "light": "https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
        "dark":  "https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{z}/{y}/{x}",
        "toner": "https://stamen-tiles.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}.png",
    }
    tile_url = tiles.get(theme, tiles["dark"])

    # i file comuni sono "index.html" o "talkmap.html"
    for fname in ("map.html", "talkmap.html"):
        fpath = pathlib.Path(folder) / fname
        if fpath.exists():
            html = fpath.read_text(encoding="utf-8")
            # rimpiazza qualunque tile Esri/OSM con il nostro (mantiene attribution ecc.)
            html2 = re.sub(
                r"L\.tileLayer\(['\"].*?/tile/\{z\}.*?\{x\}['\"],",
                f"L.tileLayer('{tile_url}',",
                html,
                flags=re.S
            )
            fpath.write_text(html2, encoding="utf-8")
            return fpath
    raise FileNotFoundError("HTML della mappa non trovato (atteso talkmap/index.html o talkmap/talkmap.html).")

# uso:
patched = set_bicolor_tiles("talkmap", theme="dark")  # 'light' / 'dark' / 'toner'
print("Mappa aggiornata:", patched)

Mappa aggiornata: talkmap/map.html


In [7]:
# Save the map
m = getorg.orgmap.create_map_obj()

m.layers = m.layers[:1]

# Aggiungi uno stile bicolore (Carto light/dark, ad esempio)
m.add_layer(TileLayer(
    url="https://cartodb-basemaps-a.global.ssl.fastly.net/light_all/{z}/{x}/{y}{r}.png",
    attribution="&copy; <a href='https://www.openstreetmap.org/copyright'>OSM</a> &copy; <a href='https://carto.com/'>CARTO</a>",
    name="Carto Light"
))

getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'

In [8]:
import os

In [9]:
! pwd

/Users/epigani/Documents/epigani.github.io


In [12]:
import folium
from folium.plugins import MarkerCluster
from pathlib import Path

def make_bicolor_map(location_dict, theme="light", outfile="talkmap/index.html"):
    """
    location_dict: dict -> {label: (lat, lon)} oppure {label: {"lat":..,"lon":..}}
    theme: 'light' | 'dark' | 'toner'
    """
    tiles = {
        "light": ("https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
                  "Tiles © Esri — Esri, HERE, Garmin, FAO, NOAA, USGS, © OpenStreetMap contributors"),
        "dark":  ("https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{z}/{y}/{x}",
                  "Tiles © Esri — Esri, HERE, Garmin, FAO, NOAA, USGS, © OpenStreetMap contributors"),
        "toner": ("https://stamen-tiles.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}.png",
                  "Map tiles by Stamen Design, © OpenStreetMap contributors")
    }
    tile_url, attr = tiles.get(theme, tiles["light"])

    pts = []
    for name, val in location_dict.items():
        try:
            if isinstance(val, dict):
                lat = float(val.get("lat"))
                lon = float(val.get("lon") or val.get("lng"))
            else:
                lat, lon = map(float, val)  # tuple
            pts.append((lat, lon, str(name)))
        except Exception:
            # salta entry malformate
            continue

    # Centro della mappa
    if pts:
        center = (sum(p[0] for p in pts)/len(pts), sum(p[1] for p in pts)/len(pts))
    else:
        center = (20, 0)

    m = folium.Map(location=center, zoom_start=2, tiles=None)
    folium.TileLayer(tiles=tile_url, attr=attr, name=f"{theme} bicolor", control=True).add_to(m)

    mc = MarkerCluster(show=False)
    for lat, lon, label in pts:
        folium.Marker(location=(lat, lon), tooltip=label, popup=label).add_to(mc)
    mc.add_to(m)

    folium.LayerControl().add_to(m)

    Path(outfile).parent.mkdir(parents=True, exist_ok=True)
    m.save(outfile)
    return m

In [13]:
# m = make_bicolor_map(location_dict, theme="light", outfile="talkmap/index.html")
m = make_bicolor_map(location_dict, theme="dark", outfile="talkmap/map.html")